In [1]:
import os
import torch, torchvision

os.chdir("/home/kishen/documents/python_projects/stanford_dogs")

from src.data import ImageDataset

In [2]:
#read data and split to train/test sets
root = "/home/kishen/documents/python_projects/stanford_dogs/data"
X = ImageDataset(root)

train_size = int(X.__len__() * .8)
test_size = X.__len__() - train_size

ran_gen= torch.Generator().manual_seed(24)
train_set, val_set = torch.utils.data.random_split(X, 
                                                  [train_size, test_size],
                                                  generator= ran_gen)



In [3]:
#calculcate trainig set metrics
from src.modeling import calculate_channel_means, calculate_channel_sds

# train_mean = calculate_channel_means(train_set, "image")
# train_sd = calculate_channel_sds(train_set, 'image')

# display(train_mean) #tensor([121.4168, 115.1198,  99.5976])
# display(train_sd) #tensor([61.0905, 59.8789, 59.7863])

In [4]:
#append transformation to train + val sets
path = "/home/kishen/documents/python_projects/stanford_dogs/data"

transform = torchvision.transforms.Compose([
    
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    #torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean= [121.3803, 115.1846,  99.6223], 
                                     std= [61.1698, 59.9492, 59.7626])

])

train_set.transform = transform
val_set.transform = transform

In [5]:
train_set.transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    RandomHorizontalFlip(p=0.5)
    Normalize(mean=[121.3803, 115.1846, 99.6223], std=[61.1698, 59.9492, 59.7626])
)

In [6]:
train_set.__getitem__(401)['image']

root = "/home/kishen/documents/python_projects/stanford_dogs/data"
test = ImageDataset(root, transform=transform)

test.__getitem__(401)['image']

/home/kishen/.virtualenvs/stanford_dogs/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor([[[-1.0200e+00, -1.1135e+00, -9.2370e-01,  ..., -5.0265e-01,
           5.9349e-02, -5.1616e-01],
         [-3.6611e-01, -5.3133e-01, -6.8775e-01,  ...,  2.8792e-02,
           2.5074e-01,  1.7443e-01],
         [ 1.7513e-01, -1.3195e-01, -5.9421e-01,  ...,  1.6390e-01,
           4.9874e-01,  5.9645e-01],
         ...,
         [ 6.4050e-01,  6.5587e-01,  2.6301e-01,  ...,  9.6822e-02,
          -5.2876e-01, -3.5771e-01],
         [ 8.2906e-01,  5.9619e-01,  2.2153e-01,  ...,  6.8766e-02,
          -2.0991e-01, -1.2618e-01],
         [ 3.3419e-01, -2.9611e-02, -1.2232e-01,  ..., -1.0677e-01,
           2.0318e-04,  4.7686e-02]],

        [[-9.8750e-01, -1.1158e+00, -9.6941e-01,  ..., -2.1095e-01,
           2.9022e-01, -2.6010e-01],
         [-3.3694e-01, -5.1390e-01, -7.0694e-01,  ...,  1.9604e-01,
           4.0452e-01,  3.6488e-01],
         [ 2.6537e-01, -9.8010e-02, -6.0305e-01,  ...,  2.3179e-01,
           5.3360e-01,  7.2629e-01],
         ...,
         [ 9.6660e-01,  1

In [7]:
#create dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

NameError: name 'train_data' is not defined

In [ ]:
# #calculcate train data stats
# from src.modeling import calculate_channel_means, calculate_channel_sds

# display(calculate_channel_means(train_data, 'image'))
# #tensor([0.4760, 0.4517, 0.3907])

# display(calculate_channel_sds(train_data, 'image'))
# #tensor([0.2362, 0.2314, 0.2307])


## Model Evaluation
We will evaluate performances using EfficientNet, Resnet18, and VGG16

In [ ]:
#init device, criterion, training loop, num_classes
from src.modeling import train_model, model_check_cuda, plot_loss, plot_acc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is:", device )

criterion = torch.nn.CrossEntropyLoss()

num_classses = len(dataset.class_labels)

In [ ]:
#init efficientNet and move to Device
efficientNet = torchvision.models.efficientnet_b0()
print(efficientNet.classifier[-1])

efficientNet.classifier[-1] = torch.nn.Linear(in_features= 1280, out_features = num_classses)

efficientNet.to(device)
model_check_cuda(efficientNet)

In [ ]:
#init optimizer and train hyperparams
opt_params = {#"params": None,
              "lr":.001,
              "momentum": .9}

train_params = {#'model': None,
                'dataloader':train_dataloader, 
                'epochs':10,
                #'optimizer': None,
                'criterion':criterion, 
                'device':device}

In [ ]:
# #Train efficientNet
# import torch.optim as optim

# opt_effNet = optim.SGD(params= efficientNet.parameters(), **opt_params)

# efficientNet_dict = {}

# efficientNet_dict = train_model(model= efficientNet,
#                                 optimizer= opt_effNet,
#                                 **train_params)


In [ ]:
#plot_acc(efficientNet_dict) 